<h5> test of download_model_function </h5>


In [3]:
from work_with_model import download_and_save_model
model = "sentence-transformers/paraphrase-albert-small-v2"

model.replace("@","@")

'sentence-transformers/paraphrase-albert-small-v2'

In [2]:
from work_with_model import load_model_bin

model_name = "sentence-transformers/paraphrase-MiniLM-L3-v2"
# model_name=model.split("/")[-1]
print(model_name)
obj = load_model_bin(model_name)

sentence-transformers/paraphrase-MiniLM-L3-v2


In [3]:
obj[1]

BertTokenizerFast(name_or_path='models/sentence-transformers/paraphrase-MiniLM-L3-v2/tokenizer', vocab_size=30522, model_max_length=512, is_fast=True, padding_side='right', truncation_side='right', special_tokens={'unk_token': '[UNK]', 'sep_token': '[SEP]', 'pad_token': '[PAD]', 'cls_token': '[CLS]', 'mask_token': '[MASK]'})

In [2]:
from work_with_model import load_model_bin

load_model_bin()

False


(BertForSequenceClassification(
   (bert): BertModel(
     (embeddings): BertEmbeddings(
       (word_embeddings): Embedding(30522, 384, padding_idx=0)
       (position_embeddings): Embedding(512, 384)
       (token_type_embeddings): Embedding(2, 384)
       (LayerNorm): LayerNorm((384,), eps=1e-12, elementwise_affine=True)
       (dropout): Dropout(p=0.1, inplace=False)
     )
     (encoder): BertEncoder(
       (layer): ModuleList(
         (0-2): 3 x BertLayer(
           (attention): BertAttention(
             (self): BertSelfAttention(
               (query): Linear(in_features=384, out_features=384, bias=True)
               (key): Linear(in_features=384, out_features=384, bias=True)
               (value): Linear(in_features=384, out_features=384, bias=True)
               (dropout): Dropout(p=0.1, inplace=False)
             )
             (output): BertSelfOutput(
               (dense): Linear(in_features=384, out_features=384, bias=True)
               (LayerNorm): LayerNor

In [10]:
from work_with_model import encode_model_bin
import torch

def encode_text(text):
    model,tokenizer = load_model_bin()
    print(model is None)

    encoding = tokenizer.encode_plus(
        text,
        add_special_tokens=True,
        return_token_type_ids=False,
        return_attention_mask=False,
        return_tensors='pt'
    )
    with torch.no_grad():
        outputs = model(**encoding)
        last_hidden_state = outputs.hidden_states[-1]
        embeddings = last_hidden_state.mean(dim=1).squeeze()
    return embeddings.numpy()


In [12]:
encode_text("nlp book")

False


TypeError: 'NoneType' object is not subscriptable

In [6]:
from dataclasses import dataclass

@dataclass
class model:
    a = 1
    b = a+1

p = model()

In [8]:
p.b

2

## new test

In [3]:
from work_with_model import transformer_ops as tops
name = "GPT_125M"
wom = tops(name=name)
# name = "sentence-transformers@paraphrase-albert-small-v2"

In [5]:
wom.setter()

Directory already exists


In [6]:
wom.download_and_save_model_pickle(name)

GPT_125M already exists no need to download again ...


In [9]:
vec = wom.encode_single_doc("the cat is strong ")

In [10]:
vec

array([-1.11646667e-01, -3.32449190e-02,  4.19502817e-02, -1.35372598e-02,
        1.25394072e-02, -5.47454767e-02, -9.21722874e-03,  1.30647756e-02,
        2.34391186e-02, -1.68136824e-02, -1.19445743e-02,  8.12968761e-02,
       -1.99614093e-02,  5.88950329e-02,  8.64244904e-03,  1.01656780e-01,
        1.35027573e-01, -3.15942317e-02, -4.71081622e-02,  7.98866674e-02,
        3.33311632e-02,  3.95192169e-02,  7.63263740e-03, -9.72219463e-03,
        6.06915988e-02,  1.23816967e-01, -1.88962836e-02,  4.94065601e-03,
        6.64008688e-03,  6.44846410e-02,  1.70861203e-02,  7.19634220e-02,
        3.16454098e-04,  6.44286796e-02, -5.19151688e-02, -6.88865557e-02,
       -1.06475890e-01,  1.02746882e-01, -8.52695294e-03,  4.72451607e-03,
        5.18697612e-02,  1.17117107e-01, -4.46294509e-02,  6.99138418e-02,
        1.90051999e-02, -6.65071160e-02, -3.49321105e-02, -7.01009631e-02,
        3.19888331e-02, -5.05961142e-02, -1.00078635e-01,  3.10247745e-02,
        1.32996336e-01, -

## encoding text function new way

In [7]:
# from transformers import AutoTokenizer, AutoModel
import torch
import torch.nn.functional as F
import transformers

#Mean Pooling - Take average of all tokens
def mean_pooling(model_output, attention_mask):
    token_embeddings = model_output.last_hidden_state
    input_mask_expanded = attention_mask.unsqueeze(-1).expand(token_embeddings.size()).float()
    return torch.sum(token_embeddings * input_mask_expanded, 1) / torch.clamp(input_mask_expanded.sum(1), min=1e-9)


#Encode text
def encode(texts):
    # Tokenize sentences
    encoded_input = tokenizer(texts, padding=True, truncation=True, return_tensors='pt')

    # Compute token embeddings
    with torch.no_grad():
        model_output = model(**encoded_input, return_dict=True)

    # Perform pooling
    embeddings = mean_pooling(model_output, encoded_input['attention_mask'])

    # Normalize embeddings
    embeddings = F.normalize(embeddings, p=2, dim=1)
    
    return embeddings



def check_transformers(string):
    if hasattr(transformers, string):
        return getattr(transformers, string)
    else:
        return None
    
AutoModel     = check_transformers("AutoModel")
AutoTokenizer = check_transformers("AutoTokenizer")

# Sentences we want sentence embeddings for
query = "How many people live in London?"
docs = ["Around 9 Million people live in London", "London is known for its financial district"]

# Load model from HuggingFace Hub
tokenizer = AutoTokenizer.from_pretrained("sentence-transformers/multi-qa-MiniLM-L6-cos-v1")
model = AutoModel.from_pretrained("sentence-transformers/multi-qa-MiniLM-L6-cos-v1")

#Encode query and docs
query_emb = encode(query)
doc_emb = encode(docs)

#Compute dot score between query and all document embeddings
scores = torch.mm(query_emb, doc_emb.transpose(0, 1))[0].cpu().tolist()

#Combine docs & scores
doc_score_pairs = list(zip(docs, scores))

#Sort by decreasing score
doc_score_pairs = sorted(doc_score_pairs, key=lambda x: x[1], reverse=True)

#Output passages & scores
for doc, score in doc_score_pairs:
    print(score, doc)


0.915637195110321 Around 9 Million people live in London
0.4947577714920044 London is known for its financial district


In [1]:
from work_with_model import transformer_ops as tops

name = "sentence-transformers@multi-qa-MiniLM-L6-cos-v1"
# name = "GPT_125M"
 
wom = tops(name=name)


Directory already exists and model also downloaded


In [2]:
tokenizer,model = wom.access_model_for_tesing()

In [5]:
import transformers

def check_transformers(string):
    if hasattr(transformers, string):
        return getattr(transformers, string)
    else:
        return None
    
check_transformers("AutoModel")
check_transformers("AutoTokenizer")

transformers.models.auto.tokenization_auto.AutoTokenizer

In [ ]:
# from transformers import AutoTokenizer, AutoModel
import torch


#Mean Pooling - Take attention mask into account for correct averaging
def mean_pooling(model_output, attention_mask):
    token_embeddings = model_output[0] #First element of model_output contains all token embeddings
    input_mask_expanded = attention_mask.unsqueeze(-1).expand(token_embeddings.size()).float()
    return torch.sum(token_embeddings * input_mask_expanded, 1) / torch.clamp(input_mask_expanded.sum(1), min=1e-9)


# Sentences we want sentence embeddings for
sentences = ['This is an example sentence', 'Each sentence is converted']
sentences = ["the cat is strong "]


# # Load model from HuggingFace Hub
# tokenizer = AutoTokenizer.from_pretrained('sentence-transformers/paraphrase-albert-small-v2')
# model = AutoModel.from_pretrained('sentence-transformers/paraphrase-albert-small-v2')


def encode_into_vector(sentences):
    # Tokenize sentences
    encoded_input = tokenizer(sentences, padding=True, truncation=True, return_tensors='pt')

    # Compute token embeddings
    with torch.no_grad():
        model_output = model(**encoded_input)

    # Perform pooling. In this case, max pooling.
    sentence_embeddings = mean_pooling(model_output, encoded_input['attention_mask'])

    print("Sentence embeddings:")
    print(sentence_embeddings)
    print(len(sentence_embeddings[0]))

encode_into_vector(sentences)

RuntimeError: expand(torch.LongTensor{[1, 6, 1]}, size=[1, 2]): the number of sizes provided (2) must be greater or equal to the number of dimensions in the tensor (3)

## gensim model

In [ ]:
!pip install gensim

     ---------------------------------------- 0.0/23.9 MB ? eta -:--:--
     --------------------------------------- 0.0/23.9 MB 991.0 kB/s eta 0:00:25
     ---------------------------------------- 0.1/23.9 MB 1.5 MB/s eta 0:00:16
     --------------------------------------- 0.1/23.9 MB 901.1 kB/s eta 0:00:27
     --------------------------------------- 0.2/23.9 MB 980.4 kB/s eta 0:00:25
     ---------------------------------------- 0.3/23.9 MB 1.2 MB/s eta 0:00:21
      --------------------------------------- 0.4/23.9 MB 1.4 MB/s eta 0:00:18
      --------------------------------------- 0.5/23.9 MB 1.5 MB/s eta 0:00:16
     - -------------------------------------- 0.6/23.9 MB 1.5 MB/s eta 0:00:16
     - -------------------------------------- 0.7/23.9 MB 1.6 MB/s eta 0:00:15
     - -------------------------------------- 0.8/23.9 MB 1.7 MB/s eta 0:00:14
     - -------------------------------------- 0.9/23.9 MB 1.7 MB/s eta 0:00:14
     - -------------------------------------- 1.0/23.9 M

In [ ]:
!pip install nltk

  Using cached nltk-3.8.1-py3-none-any.whl (1.5 MB)
  Using cached click-8.1.3-py3-none-any.whl (96 kB)
  Using cached joblib-1.2.0-py3-none-any.whl (297 kB)


In [ ]:
from gensim.models.doc2vec import Doc2Vec, TaggedDocument
from gensim.utils import tokenize
from file_metadata import get_all_metadata

# define a list of documents
documents = get_all_metadata

# tokenize each document
tokenized_docs = [ list(tokenize(doc.lower())) for doc in documents]

# create a list of TaggedDocument objects
tagged_docs = [TaggedDocument(words=doc, tags=[str(i)]) for i, doc in enumerate(tokenized_docs)]

# train a Doc2Vec model
model = Doc2Vec(vector_size=50, min_count=2, epochs=40)
model.build_vocab(tagged_docs)
model.train(tagged_docs, total_examples=model.corpus_count, epochs=model.epochs)

# get the document vector for the first document
doc_vector = model.infer_vector(tokenized_docs[0])
print(doc_vector)


[ 0.0002148   0.00935951  0.00167974  0.00904912 -0.00824874  0.00864258
  0.00969176 -0.0079954   0.00134886  0.00818412  0.00885907 -0.00500215
 -0.00490618 -0.00961742  0.00423111 -0.002891    0.00361814  0.00456762
  0.00208496 -0.00475434  0.00592477  0.00973361 -0.00728942  0.00339555
  0.00282166  0.0060231   0.00781083  0.00559872 -0.00594684 -0.00517285
  0.00821882 -0.0096197   0.00495436  0.00889858  0.00207776  0.00326364
  0.00310452  0.00227818  0.00533976 -0.00350923  0.00437772 -0.00365946
 -0.00487915 -0.00158954  0.00970974 -0.00068267 -0.00135474 -0.00738559
  0.00331212 -0.00108221]


In [ ]:
import test

test.db_obj

Directory already exists


## downloading cos-v1 sent transformer

In [ ]:
from work_with_model import transformer_ops

name = "sentence-transformers/multi-qa-MiniLM-L6-cos-v1"

trobj = transformer_ops(name)

Directory already exists but model not downloaded


In [ ]:
trobj.get_data_for_testing_purpose()

False
sentence-transformers@multi-qa-MiniLM-L6-cos-v1
d:\pendrive\filesearch2.0\models\sentence-transformers@multi-qa-MiniLM-L6-cos-v1\model.pkl
d:\pendrive\filesearch2.0\models\sentence-transformers@multi-qa-MiniLM-L6-cos-v1\tokenizer.pkl


In [ ]:
trobj.download_and_save_model_pickle(name)

c:\Users\shree\AppData\Local\Programs\Python\Python311\Lib\site-packages\huggingface_hub\file_download.py:133: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\shree\.cache\huggingface\hub. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to see activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


Some weights of BertForSequenceClassification were not initialized from the model checkpoint at sentence-transformers/multi-qa-MiniLM-L6-cos-v1 and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
